In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from plotly import graph_objects as go
import plotly.io as pio
import json
from plotly.offline import plot

In [35]:
data = pd.read_csv("./sample.csv")
data['date'] = pd.to_datetime(data['date'])

In [36]:
data

,date,CSI300_Close,CSI1000_Close,LCVI_Close,SCGI_Close,WASI_Close,CSI300_PB,CSI1000_PB,LCVI_PB,SCGI_PB,WASI_PB
0,2025-03-03,3888.4738,6273.6699,7793.0761,4785.4078,5144.8037,1.3453,2.1246,0.8632,2.3943,1.5877
1,2025-03-04,3885.2208,6357.6277,7785.7625,4815.4691,5176.5321,1.3428,2.1492,0.8581,2.4326,1.5944
2,2025-03-05,3902.5706,6398.2370,7856.5366,4814.2034,5201.0337,1.3524,2.1633,0.8680,2.4320,1.6038
3,2025-03-06,3956.2369,6534.9711,7884.2066,4900.8355,5285.3792,1.3647,2.2030,0.8676,2.4712,1.6237
4,2025-03-07,3944.0143,6507.7645,7878.6584,4882.2190,5262.8346,1.3615,2.1946,0.8677,2.4642,1.6182
...,...,...,...,...,...,...,...,...,...,...,...
105,2025-08-04,4070.7023,6739.6936,8413.6169,5019.0162,5601.6675,1.3883,2.3004,0.9457,2.7414,1.6903
106,2025-08-05,4103.4500,6787.4806,8508.1966,5046.7534,5645.2615,1.4017,2.3179,0.9568,2.7572,1.7054
107,2025-08-06,4113.4852,6861.3091,8517.3285,5104.8154,5680.0500,1.4042,2.3416,0.9576,2.7878,1.7139
108,2025-08-07,4114.6660,6862.1497,8529.6940,5095.8068,5679.9163,1.4067,2.3421,0.9613,2.7821,1.7148


#### **动量因子风格择时信号：**

##### 需要数据： 沪深300指数和中证1000指数日收盘价 
##### 信号逻辑： 取前第一日和第十一日指数收盘价计算过去十天指数收益率：
#####        (1) 沪深300指数收益率 > 中证1000指数收益率： 当天开盘买入沪深300指数,收盘平仓
#####        (2) 沪深300指数收益率 < 中证1000指数收益率： 当天开盘买入中证1000指数，收盘平仓
#####        (3) 若两者收益率均小于0：当天空仓

In [37]:

data["CSI300_Close_1"] = data['CSI300_Close'].shift()
data["CSI300_Close_11"] = data['CSI300_Close'].shift(11)
data["CSI1000_Close_1"] = data['CSI1000_Close'].shift()
data["CSI1000_Close_11"] = data['CSI1000_Close'].shift(11)
data['10_days_CSI300_rtn'] = data['CSI300_Close_1']/data['CSI300_Close_11']-1
data['10_days_CSI1000_rtn'] = data['CSI1000_Close_1']/data['CSI1000_Close_11']-1

In [38]:
data

,date,CSI300_Close,CSI1000_Close,LCVI_Close,SCGI_Close,WASI_Close,CSI300_PB,CSI1000_PB,LCVI_PB,SCGI_PB,WASI_PB,CSI300_Close_1,CSI300_Close_11,CSI1000_Close_1,CSI1000_Close_11,10_days_CSI300_rtn,10_days_CSI1000_rtn
0,2025-03-03,3888.4738,6273.6699,7793.0761,4785.4078,5144.8037,1.3453,2.1246,0.8632,2.3943,1.5877,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-03-04,3885.2208,6357.6277,7785.7625,4815.4691,5176.5321,1.3428,2.1492,0.8581,2.4326,1.5944,3888.4738,NaN,6273.6699,NaN,NaN,NaN
2,2025-03-05,3902.5706,6398.2370,7856.5366,4814.2034,5201.0337,1.3524,2.1633,0.8680,2.4320,1.6038,3885.2208,NaN,6357.6277,NaN,NaN,NaN
3,2025-03-06,3956.2369,6534.9711,7884.2066,4900.8355,5285.3792,1.3647,2.2030,0.8676,2.4712,1.6237,3902.5706,NaN,6398.2370,NaN,NaN,NaN
4,2025-03-07,3944.0143,6507.7645,7878.6584,4882.2190,5262.8346,1.3615,2.1946,0.8677,2.4642,1.6182,3956.2369,NaN,6534.9711,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,2025-08-04,4070.7023,6739.6936,8413.6169,5019.0162,5601.6675,1.3883,2.3004,0.9457,2.7414,1.6903,4054.9286,4058.5486,6670.4720,6552.0741,-0.000892,0.018070
106,2025-08-05,4103.4500,6787.4806,8508.1966,5046.7534,5645.2615,1.4017,2.3179,0.9568,2.7572,1.7054,4070.7023,4085.6072,6739.6936,6612.2589,-0.003648,0.019272
107,2025-08-06,4113.4852,6861.3091,8517.3285,5104.8154,5680.0500,1.4042,2.3416,0.9576,2.7878,1.7139,4103.4500,4118.9582,6787.4806,6637.1035,-0.003765,0.022657
108,2025-08-07,4114.6660,6862.1497,8529.6940,5095.8068,5679.9163,1.4067,2.3421,0.9613,2.7821,1.7148,4113.4852,4119.7685,6861.3091,6607.2200,-0.001525,0.038456


In [39]:
def signal1_row(row):
    if (row['10_days_CSI300_rtn'] * row['10_days_CSI1000_rtn'] > 0) & (row['10_days_CSI300_rtn'] < 0):
        row['Signal1'] = 0
        row['Signal1_day_rtn'] = 1
        row['Position1'] = '空仓'
    elif row['10_days_CSI300_rtn'] > row['10_days_CSI1000_rtn']:
        row['Signal1'] = 1
        row['Signal1_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position1'] = '满仓沪深300指数'
    else: 
        row['Signal1'] = -1
        row['Signal1_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position1'] = '满仓中证1000指数'
    return row

data = data.apply(signal1_row, axis = 1)

In [8]:
# fig1 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal1_rtn'],
#                  text = data['Position1'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig1.update_layout(template='simple_white',
#                    title='<b>动量因子风格择时信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig1.show()

#### **拥挤度动量因子风格择时信号：**

##### 需要数据： 大盘价值指数PB,小盘成长指数PB，万得全A指数PB 
##### 信号逻辑： 取前第一日和第五日指数PB计算指标：
#####           R1 = (大盘价值指数PB-小盘成长指数PB)/万得全A指数PB
#####        (1) R1(t-1) > R1(t-5)： 当天开盘买入沪深300指数,收盘平仓
#####        (2) R1(t-1) < R1(t-5)： 当天开盘买入中证1000指数，收盘平仓

In [46]:
data['LCVI_PB_1'] = data['LCVI_PB'].shift()
data['LCVI_PB_5'] = data['LCVI_PB'].shift(5)
data['SCGI_PB_1'] = data['SCGI_PB'].shift()
data['SCGI_PB_5'] = data['SCGI_PB'].shift(5)
data['WASI_PB_1'] = data['WASI_PB'].shift()
data['WASI_PB_5'] = data['WASI_PB'].shift(5)

In [47]:
def signal2_row(row):
    if (row['LCVI_PB_1'] - row['SCGI_PB_1'])/row['WASI_PB_1'] > (row['LCVI_PB_5'] - row['SCGI_PB_5'])/row['WASI_PB_5']:
        row['Signal2'] = 1
        row['Signal2_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position2'] = '满仓沪深300指数'
    else: 
        row['Signal2'] = -1
        row['Signal2_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position2'] = '满仓中证1000指数'
    return row

data = data.apply(signal2_row, axis = 1)

In [11]:
# fig2 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal2_rtn'],
#                  text = data['Position2'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig2.update_layout(template='simple_white',
#                    title='<b>拥挤度动量因子风格择时信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081",
#                    ),
#                    width=2000, 
#                    height=500)
# fig2.show()

#### **相对强弱动量因子风格择时信号（大盘价值指数和小盘成长指数）**

##### 需要数据： 大盘价值指数、小盘成长指数、万得全A指数收盘价 
##### 信号逻辑： 取前第一日和第二日指数收盘价计算指标：
#####           d1 = 大盘价值指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####           d2 = 小盘成长指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####        (1) d1，d2小于0.2%： 空仓
#####        (2) d1 > d2: 择时沪深300指数
#####        (3) d1 < d2: 择时中证1000指数

In [48]:
data['LCVI_Close_1'] = data['LCVI_Close'].shift()
data['LCVI_Close_2'] = data['LCVI_Close'].shift(2)
data['SCGI_Close_1'] = data['SCGI_Close'].shift()
data['SCGI_Close_2'] = data['SCGI_Close'].shift(2)
data['WASI_Close_1'] = data['WASI_Close'].shift()
data['WASI_Close_2'] = data['WASI_Close'].shift(2)
data['LCVI_rtn_1'] = data['LCVI_Close_1']/data['LCVI_Close_2']
data['SCGI_rtn_1'] = data['SCGI_Close_1']/data['SCGI_Close_2']
data['WASI_rtn_1'] = data['WASI_Close_1']/data['WASI_Close_2']

In [49]:
def signal3_row(row):
    if (row['LCVI_rtn_1'] - row['WASI_rtn_1'] < 0.002) & (row['SCGI_rtn_1'] - row['WASI_rtn_1'] < 0.002):
        row['Signal3'] = 0
        row['Signal3_day_rtn'] = 1
        row['Position3'] = '空仓'
    elif row['LCVI_rtn_1'] > row['SCGI_rtn_1']:
        row['Signal3'] = 1
        row['Signal3_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position3'] = '满仓沪深300指数'
    else: 
        row['Signal3'] = -1
        row['Signal3_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position3'] = '满仓中证1000指数'
    return row

data = data.apply(signal3_row, axis = 1)

In [14]:
# fig3 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal3_rtn'],
#                  text = data['Position3'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig3.update_layout(template='simple_white',
#                    title='<b>相对强弱动量因子风格择时信号(大盘价值指数和小盘成长指数)<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig3.show()

#### **相对强弱动量因子风格择时信号（沪深300指数和中证1000指数）**

##### 需要数据： 沪深300指数、中证1000指数、万得全A指数收盘价 
##### 信号逻辑： 取前第一日和第二日指数收盘价计算指标：
#####           d1 = 沪深300指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####           d2 = 中证1000指数日收益率(t-1)-万得全A指数日收益率(t-1)
#####        (1) d1，d2小于0.2%： 空仓
#####        (2) d1 > d2: 择时沪深300指数
#####        (3) d1 < d2: 择时中证1000指数

In [50]:
data['CSI300_Close_2'] = data['CSI300_Close'].shift(2)
data['CSI1000_Close_2'] = data['CSI1000_Close'].shift(2)
data['CSI300_rtn_1'] = data['CSI300_Close_1']/data['CSI300_Close_2']
data['CSI1000_rtn_1'] = data['CSI1000_Close_1']/data['CSI1000_Close_2']

In [51]:
def signal4_row(row):
    if (row['CSI300_rtn_1'] - row['WASI_rtn_1'] < 0.002) & (row['CSI1000_rtn_1'] - row['WASI_rtn_1'] < 0.002):
        row['Signal4'] = 0
        row['Signal4_day_rtn'] = 1
        row['Position4'] = '空仓'
    elif row['CSI300_rtn_1'] > row['CSI1000_rtn_1']:
        row['Signal4'] = 1
        row['Signal4_day_rtn'] = row['CSI300_Close']/row['CSI300_Close_1']
        row['Position4'] = '满仓沪深300指数'
    else: 
        row['Signal4'] = -1
        row['Signal4_day_rtn'] = row['CSI1000_Close']/row['CSI1000_Close_1']
        row['Position4'] = '满仓中证1000指数'
    return row

data = data.apply(signal4_row, axis = 1)

In [17]:
# fig4 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal4_rtn'],
#                  text = data['Position4'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig4.update_layout(template='simple_white',
#                    title='<b>相对强弱动量因子风格择时信号(沪深300指数和中证1000指数)<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig4.show()

#### **多维动量复合风格择时对冲信号**

##### 信号逻辑： 根据以上四个信号计算指标
#####          c1 = 择时沪深300指数信号数量
#####          c2 = 择时中证1000指数信号数量
#####         (1) c1 - c2 > 2: 7.5%多IF + 7.5%空IM
#####         (2) 0 < c1 - c2 < 2: 5%多IF + 5%空IM
#####         (3) c1 - c2 = 0: 空仓
#####         (4) -2 < c1 - c2 < 0: 5%多IM + 5%空IF
#####         (5) c1 - c2 < -2: 7.5%多IM + 7.5%空IF

In [52]:
margin = 0.12
data['Signal_all'] = data['Signal1'] + data['Signal2'] + data['Signal3'] + data['Signal4']

In [53]:
def signal5_row(row):
    if (row['Signal_all'] > 2):
        row['Signal_all_day_rtn'] = (row['CSI300_Close']/row['CSI300_Close_1']-1)*0.075/margin-(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.075/margin+1
        row['Position5'] = '7.5%多IF + 7.5%空IM'
    elif row['Signal_all'] > 0:
        row['Signal_all_day_rtn'] = (row['CSI300_Close']/row['CSI300_Close_1']-1)*0.05/margin-(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.05/margin+1
        row['Position5'] = '5%多IF + 5%空IM'
    elif row['Signal_all'] == 0:
        row['Signal_all_day_rtn'] = 1
        row['Position5'] = '空仓'
    elif row['Signal_all'] > -2:
        row['Signal_all_day_rtn'] = -(row['CSI300_Close']/row['CSI300_Close_1']-1)*0.05/margin+(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.05/margin+1
        row['Position5'] = '5%多IM + 5%空IF'
    else:
        row['Signal_all_day_rtn'] = -(row['CSI300_Close']/row['CSI300_Close_1']-1)*0.075/margin+(row['CSI1000_Close']/row['CSI1000_Close_1']-1)*0.075/margin+1
        row['Position5'] = '7.5%多IM + 7.5%空IF'
    return row

data = data.apply(signal5_row, axis = 1)

In [20]:
# fig5 = go.Figure(data = go.Scatter(
#                  x = data['date'],
#                  y = data['Signal5_rtn'],
#                  text = data['Position5'],
#                  hovertemplate= '<b>Date</b>: %{x|%Y-%m-%d}<br>'+
#                  '<b>Position</b>: %{text}',
#                  name = '',
#                  mode = 'lines',
#                  line=dict(color="#0d5081", width=2)))
# fig5.update_layout(template='simple_white',
#                    title='<b>多维动量复合风格择时对冲信号<b>',
#                    title_font=dict(
#                    family='KaiTi',  # 楷体字体
#                    size=24,
#                    color="#0d5081"
#                    ),
#                    width=2000, 
#                    height=500)
# fig5.show()

In [21]:
data = data.loc[data['date'] > '2010-01-01',:].reset_index(drop=True)

In [54]:
data

,date,CSI300_Close,CSI1000_Close,LCVI_Close,SCGI_Close,WASI_Close,CSI300_PB,CSI1000_PB,LCVI_PB,SCGI_PB,...,CSI300_Close_2,CSI1000_Close_2,CSI300_rtn_1,CSI1000_rtn_1,Signal4,Signal4_day_rtn,Position4,Signal_all,Signal_all_day_rtn,Position5
0,2025-03-03,3888.4738,6273.6699,7793.0761,4785.4078,5144.8037,1.3453,2.1246,0.8632,2.3943,...,NaN,NaN,NaN,NaN,-1,NaN,满仓中证1000指数,-4,NaN,7.5%多IM + 7.5%空IF
1,2025-03-04,3885.2208,6357.6277,7785.7625,4815.4691,5176.5321,1.3428,2.1492,0.8581,2.4326,...,NaN,NaN,NaN,NaN,-1,1.013383,满仓中证1000指数,-4,1.008887,7.5%多IM + 7.5%空IF
2,2025-03-05,3902.5706,6398.2370,7856.5366,4814.2034,5201.0337,1.3524,2.1633,0.8680,2.4320,...,3888.4738,6273.6699,0.999163,1.013383,-1,1.006387,满仓中证1000指数,-3,1.001201,7.5%多IM + 7.5%空IF
3,2025-03-06,3956.2369,6534.9711,7884.2066,4900.8355,5285.3792,1.3647,2.2030,0.8676,2.4712,...,3885.2208,6357.6277,1.004466,1.006387,0,1.000000,空仓,-1,1.003175,5%多IM + 5%空IF
4,2025-03-07,3944.0143,6507.7645,7878.6584,4882.2190,5262.8346,1.3615,2.1946,0.8677,2.4642,...,3902.5706,6398.2370,1.013752,1.021371,-1,0.995837,满仓中证1000指数,-3,0.999329,7.5%多IM + 7.5%空IF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,2025-08-04,4070.7023,6739.6936,8413.6169,5019.0162,5601.6675,1.3883,2.3004,0.9457,2.7414,...,4075.5912,6661.1862,0.994930,1.001394,-1,1.010377,满仓中证1000指数,-4,1.004055,7.5%多IM + 7.5%空IF
106,2025-08-05,4103.4500,6787.4806,8508.1966,5046.7534,5645.2615,1.4017,2.3179,0.9568,2.7572,...,4054.9286,6670.4720,1.003890,1.010377,-1,1.007090,满仓中证1000指数,-2,0.999404,7.5%多IM + 7.5%空IF
107,2025-08-06,4113.4852,6861.3091,8517.3285,5104.8154,5680.0500,1.4042,2.3416,0.9576,2.7878,...,4070.7023,6739.6936,1.008045,1.007090,0,1.000000,空仓,-1,1.003513,5%多IM + 5%空IF
108,2025-08-07,4114.6660,6862.1497,8529.6940,5095.8068,5679.9163,1.4067,2.3421,0.9613,2.7821,...,4103.4500,6787.4806,1.002446,1.010877,-1,1.000123,满仓中证1000指数,-4,0.999897,7.5%多IM + 7.5%空IF


In [23]:
data['Signal1_Value'] = np.cumprod(data['Signal1_day_rtn']) * 500000
data['Signal2_Value'] = np.cumprod(data['Signal2_day_rtn']) * 500000
data['Signal3_Value'] = np.cumprod(data['Signal3_day_rtn']) * 500000
data['Signal4_Value'] = np.cumprod(data['Signal4_day_rtn']) * 500000
data['Signal5_Value'] = np.cumprod(data['Signal_all_day_rtn']) * 500000

In [24]:
column_list = ['date','Signal1_Value','Position1','Signal2_Value','Position2','Signal3_Value','Position3','Signal4_Value','Position4','Signal5_Value','Position5']
data = data.loc[:,column_list]

In [25]:
data

,date,Signal1_Value,Position1,Signal2_Value,Position2,Signal3_Value,Position3,Signal4_Value,Position4,Signal5_Value,Position5
0,2025-08-01,NaN,满仓中证1000指数,NaN,满仓中证1000指数,NaN,满仓中证1000指数,NaN,满仓中证1000指数,NaN,7.5%多IM + 7.5%空IF
1,2025-08-04,505188.658314,满仓中证1000指数,505188.658314,满仓中证1000指数,505188.658314,满仓中证1000指数,505188.658314,满仓中证1000指数,502027.284308,7.5%多IM + 7.5%空IF
2,2025-08-05,508770.638719,满仓中证1000指数,508770.638719,满仓中证1000指数,508770.638719,满仓中证1000指数,508770.638719,满仓中证1000指数,501727.841317,7.5%多IM + 7.5%空IF
3,2025-08-06,514304.617424,满仓中证1000指数,514304.617424,满仓中证1000指数,510014.863729,满仓沪深300指数,508770.638719,空仓,503490.496151,5%多IM + 5%空IF
4,2025-08-07,514367.626459,满仓中证1000指数,514367.626459,满仓中证1000指数,510077.347212,满仓中证1000指数,508832.969769,满仓中证1000指数,503438.717570,7.5%多IM + 7.5%空IF


In [26]:
data.to_csv('output.csv', index=False)